# One Portfolio to Rule Them All!

This notebook deals with the collection of financial asset data (stocks and cryptocurrency). This must be run in an environment that is compatible with alpacas API.

1. Collect data from APIs
2. Save to csv
3. Collect all data in SQL Database

In [1]:
# Import all necessary libraries
import os
from pathlib import Path
from dotenv import load_dotenv
from ft_logger import *
from PolygonIO import PolygonIO
import pandas as pd
import alpaca_trade_api as tradeapi
import datetime
from dateutil.relativedelta import relativedelta

## Collect Data From APIs

In [2]:
# start logging
start_logging()

# Import data from API/csv into Pandas DataFrame
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key=os.getenv('APCA_API_KEY_ID')
alpaca_secret_key=os.getenv('APCA_API_SECRET_KEY')
alpaca_endpoint=os.getenv('APCA_API_BASE_URL')

# Create the Alpaca API object
alpaca = tradeapi.REST()

# Format current and previous date as ISO format
date = datetime.date.today()
date_fmt = date.strftime('%Y-%m-%d')
today = pd.Timestamp(date_fmt, tz='America/New_York').isoformat()

# Set start date of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
five_yrs_ago = date - relativedelta(years=5)
five_yrs_ago = five_yrs_ago.strftime('%Y-%m-%d')
start_date = pd.Timestamp(five_yrs_ago, tz='America/New_York').isoformat()

# Set the tickers
tickers = ['AAPL', 'MSFT', 'PFE', 'DIS', 'DJIA']

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
# The current day may be a day when the markets are closed (weekend, holiday, etc.)
# So, if the retrieved portfolio is empty, let's try the previous day.
tickers_df = alpaca.get_bars(tickers, timeframe, start=start_date, end=today).df.dropna()

while tickers_df.empty:
    date -= relativedelta(days=1)
    log.warn(f"End date {today} is invalid, trying prior day {date}")
    date_fmt = date.strftime('%Y-%m-%d')
    today = pd.Timestamp(date_fmt, tz='America/New_York').isoformat()
    tickers_df = alpaca.get_bars(tickers, timeframe, start=start_date, end=today, adjustment='split').df

# change timestamp to date only
tickers_df.index=tickers_df.index.map(lambda x: x.date)

# Find any stock splits that may have occurred for each stock within the period
pg = PolygonIO('INSERT_POLYGONIO_API_KEY_HERE', tickers, five_yrs_ago, date_fmt)
splits = pg.get_splits()
tickers_df.head()

,open,high,low,close,volume,trade_count,vwap,symbol
2018-04-18,177.8100,178.8200,176.8800,177.84,20757396,136339,177.949113,AAPL
2018-04-19,174.9600,175.3900,172.6600,172.80,34822915,230448,173.639715,AAPL
2018-04-20,170.5950,171.2184,165.4300,165.72,65552591,402201,167.067748,AAPL
2018-04-23,166.8348,166.9200,164.0900,165.24,36542216,238213,165.586598,AAPL
2018-04-24,165.6700,166.3300,161.2201,162.94,33695538,258898,163.622161,AAPL


In [3]:
# Apply stock splits to the data
for split_exec in splits:
    for split in split_exec:
        adjusted_close_df = tickers_df.loc[(tickers_df.index >= datetime.datetime.strptime(split['time'], '%Y-%m-%d').date()) & (tickers_df['symbol'] == split['ticker'])]['close'].mul(split['split_factor'])
        tickers_df.loc[(tickers_df.index >= datetime.datetime.strptime(split['time'], '%Y-%m-%d').date()) & (tickers_df['symbol'] == split['ticker']), ['close']] = adjusted_close_df
log.info('Stock closing prices adjusted for splits.')

In [4]:
# Review and clean data
tickers_df_list = [tickers_df[tickers_df['symbol'] == ticker].drop('symbol', axis='columns') for ticker in tickers]
stocks_df = pd.concat(tickers_df_list, axis='columns', join='inner', keys=tickers).dropna()
display(stocks_df.head())
display(stocks_df.tail())
log.info('Stocks data cleaned.')

AAPL                                                   \
               open    high       low   close     volume trade_count   
2022-02-24  152.580  162.85  152.0000  650.04  141258414     1383835   
2022-02-25  163.780  165.12  160.8738  659.40   87680902      768624   
2022-02-28  163.060  165.42  162.4300  660.48   90547539      743310   
2022-03-01  164.695  166.60  161.9700  652.80   79455454      701957   
2022-03-02  164.390  167.36  162.9500  666.24   76135254      631927   

                           MSFT                   ...       DIS              \
                  vwap     open    high      low  ...    volume trade_count   
2022-02-24  158.018049  272.510  295.16  271.520  ...  12449572      190450   
2022-02-25  163.788661  295.185  297.62  291.655  ...   9677173      125585   
2022-02-28  164.274417  294.310  299.14  293.000  ...  10443190      118442   
2022-03-01  164.167482  296.400  299.97  292.150  ...   8494450      129582   
2022-03-02  165.810466  295.360  301.44  293.698  ...   7516010      110441   

                         DJIA                                                \
                  vwap   open     high      low    close volume trade_count   
2022-02-24  145.726019  23.78  24.0200  23.6500  24.0200  22738          52   
2022-02-25  149.054607  24.41  24.5814  24.1900  24.5814  26826         172   
2022-02-28  148.079387  24.67  24.6700  24.4400  24.5981   4076         113   
2022-03-01  145.762479  24.78  24.7800  24.2200  24.2721   8225         107   
2022-03-02  147.446804  24.65  24.7100  24.4198  24.6600  14268         123   

                       
                 vwap  
2022-02-24  23.769930  
2022-02-25  24.275010  
2022-02-28  24.535060  
2022-03-01  24.338157  
2022-03-02  24.555496  

[5 rows x 35 columns]

AAPL                                                            \
              open    high     low   close    volume trade_count        vwap   
2023-04-12  161.22  162.06  159.78  640.40  50164302      515535  160.883893   
2023-04-13  161.63  165.80  161.42  662.24  68436844      595635  164.353317   
2023-04-14  164.59  166.32  163.83  660.84  48280881      476387  165.025855   
2023-04-17  165.09  165.39  164.03  660.92  40669651      474137  164.807277   
2023-04-18  166.10  167.41  165.65  664.12  23425922      309880  166.344854   

              MSFT                   ...      DIS                          \
              open     high     low  ...   volume trade_count        vwap   
2023-04-12  284.79  287.010  281.96  ...  9298105      118951   99.327012   
2023-04-13  283.59  289.900  283.17  ...  8744879       95928  100.426837   
2023-04-14  287.00  288.480  283.69  ...  7105675       94736   99.621022   
2023-04-17  289.93  291.600  286.16  ...  6542098       88864   99.937498   
2023-04-18  291.57  291.755  287.01  ...  3745914       51439  100.299306   

             DJIA                                                      
             open   high     low  close volume trade_count       vwap  
2023-04-12  21.87  21.91  21.820  21.91  25188         278  21.858299  
2023-04-13  21.81  21.93  21.750  21.84  59568         458  21.837493  
2023-04-14  21.84  21.87  21.795  21.80  37136         328  21.851132  
2023-04-17  21.95  21.99  21.810  21.85  25439         353  21.881546  
2023-04-18  22.00  22.00  21.865  21.93  21224         162  21.899874  

[5 rows x 35 columns]

In [5]:
# save to csv
output_file = Path('./stocks_data.csv')
stocks_df.to_csv(output_file)
log.info(f"Output written to {output_file}")